In [ ]:
# Imports

import librosa
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd
import time

# for RAM managment
import gc
import sys

# import foote
#!pip install import_ipynb
import import_ipynb
# Imports for my modules
%cd "/content/drive/MyDrive/Colab Notebooks"
import Foote_modular as fm


# Optimizer

1. Randomly fixes all parameters except one, and then proceed to optimize that one.
2. Once a parameter is maximized, we fix that one. 
3. Then we proceed to maximize a second one.
4. Once all are maximized, we start again until the parameters are stable.







## 1. The file is loaded and some features are calculated.


In [ ]:
#audio_filename = '/content/drive/MyDrive/TFG/Marco Mezquida DB/test.wav'
audio_filename = '/content/drive/MyDrive/TFG/synthetic_data/mixed.wav'
boundaries_file = '/content/drive/MyDrive/TFG/synthetic_data/boundaries_mixed.txt'


signal, sample_rate = librosa.load(audio_filename, mono = True)
duration = len(signal) / sample_rate 


## 2. Points where the boundarie should be detected

In [ ]:
# segment_boundaries = [10, 20, 30, 40, 50, 54]   # in sec        FOR THE test.wav FILE
with open(boundaries_file) as file:
    segment_boundaries = [round(float(line.rstrip('\n')), 3) for line in file]

margin = 0.25           # in sec; 200 ms 
len(segment_boundaries)

60

## 2.1 Evaluation sampled signal

In [ ]:
annotation_vector = []

duration_ms = int(np.ceil(duration*1000))# we need to process it in miliseconds
periode = int((margin*1000)*2)
#print(str(periode) + 'ms')
#print(str(duration_ms) + 'ms')

# we want the annotated boundaries to be in the center of the periodes, therefore we need the first periode to be 250ms and all the others 500ms.

for i in range(0, duration_ms, periode):
  if True in (i-periode/2 <= x*1000 < i + periode/2 for x in segment_boundaries):
    annotation_vector.append(True)
  else:
    annotation_vector.append(False)



## 3.    Optimization algorithm.



# MARGINAL OPTIMITZATION - manual
  First implementation only computes - for plotting and graphical check

It should be able to optimize by itself

In [ ]:
results_df = pd.DataFrame(columns = ['FFT size', 'N Mel Filters', 'N Mel Coefficients', 
                                     'Size Kernel', 'Offset in filter', 'Median filter size', 
                                     'Precision', 'Recall', 'Specificity', 'False Positive Rate', 
                                     'F-score', 'Confusion Matrix'])


In [ ]:

#@title

def Foote(R_FFT_size, R_num_mel_filt, R_num_mel_coefficients, R_kernel_size, R_offset, R_median_filter_size):
  a = 0
  start_time = time.time()


  for fft_iter in R_FFT_size:
    # Constant parameters
    w = np.hamming(fft_iter)    # Hamming window


    for filt_iter in R_num_mel_filt:
      S = librosa.feature.melspectrogram(signal, sr = sample_rate, n_fft = fft_iter, window = w, n_mels = filt_iter)
      db_S = librosa.amplitude_to_db(S, ref=np.max)


      for coef_iter in R_num_mel_coefficients:
        mfcc = librosa.feature.mfcc(S = db_S, n_mfcc = coef_iter)

        D = scipy.spatial.distance.pdist(mfcc.T, metric = "euclidean")
        SSM = scipy.spatial.distance.squareform(D)
        SSM /= D.max()  


        for kern_iter in R_kernel_size:
          gauss_size = kern_iter

          # Gaussian vector
          g = scipy.signal.gaussian(M = gauss_size, std = gauss_size/3, sym=True)
          # 3D Gaussian 
          G = np.dot(g.reshape(-1, 1), g.reshape(1, -1))
          # dividing the 3D Gaussian matrix into 4 squares and multiplying -1 and +1
          G[gauss_size//2:, :gauss_size//2] = -G[gauss_size//2:, :gauss_size//2]
          G[:gauss_size//2, gauss_size//2:] = -G[:gauss_size//2, gauss_size//2:]


          novelty_curve = np.zeros(SSM.shape[1])

          N = SSM.shape[0]
          L = G.shape[0]

          for i in range( L//2, N - L//2 + 1):
            novelty_curve[i] = np.sum(SSM[i - L//2:i + L//2, i - L//2:i + L//2] * G)
            # If this is carashing, your L is an odd number, therefore the broadcast crahes since // retrns a rounded down number
                
          novelty_curve = np.abs(novelty_curve)

          # Normalize
          novelty_curve += novelty_curve.min()
          novelty_curve /= novelty_curve.max()

          #frames = range(len(novelty_curve))

          time_axis = np.linspace(0, duration, len(novelty_curve))


          for offset_iter in R_offset:


            for mfilter_iter in R_median_filter_size:

              threshold = scipy.ndimage.filters.median_filter(novelty_curve, size = mfilter_iter) + novelty_curve.mean()*offset_iter
              peaks = []

              for i in range(1, novelty_curve.shape[0] - 1):
                  # is it a peak?
                if novelty_curve[i - 1] < novelty_curve[i] and novelty_curve[i] > novelty_curve[i + 1]:
                  # is it above the threshold?
                  if novelty_curve[i] > threshold[i]:
                    peaks.append(i)
                          
              peaks = np.array(peaks)*(duration/len(novelty_curve))


              # SCORING COMPUTATION
              
              # We create a vector like the evaluation vector, but allocating the boundaries estimated.
              estimation_vector = [False] * len(annotation_vector)

              # First shorter period
              if True in (0 <= x*1000 < periode//2 for x in peaks):
                estimation_vector[0] = True

              # all the rest
              for i in range(0, duration_ms, periode):# we need to process it in miliseconds
                if True in (i-periode/2 <= x*1000 < i + periode/2 for x in peaks):
                  estimation_vector[i//periode] = True
              

              
              # Using the confusion matrix and its metrics, we asign a score
              # True Positive   ->   An annotated boundarie aligns with one (or more) peaks       ->  True - True     ->  AND
              # True Negative   ->   A period without annotations aligns with NO peaks            ->  False - False   ->  NOT(OR)
              # False Negative  ->   An annotated boundarie aligns with NO peaks                  ->  True - False    ->  Abjunction (Material nonimplication)
              # False Positive  ->   A period without annotations aligns with a peak              ->  False - True    ->  N peaks - TP)
            
              tp_vector = [False] * len(annotation_vector)
              tn_vector = [False] * len(annotation_vector)
              fn_vector = [False] * len(annotation_vector)
              fp_vector = [False] * len(annotation_vector)
              for i in range(0, len(annotation_vector)):
                tp_vector[i] = annotation_vector[i] and estimation_vector[i]
                tn_vector[i] = not(annotation_vector[i] or estimation_vector[i])
                fn_vector[i] = annotation_vector[i] and not(estimation_vector[i])
                fp_vector[i] = estimation_vector[i] and not(annotation_vector[i])
              

              TP = sum(tp_vector)
              TN = sum(tn_vector)
              FN = sum(fn_vector)
              FP = sum(fp_vector)

              # FN = [(annotation_vector[i] and not(estimation_vector[i])) for i in range(0, len(annotation_vector))]

              if len(peaks) == 0:
                recall = 0
                TNR = 0
                precision = 0
                false_positive_rate = 1
                Fscore = 0
              else:
                recall = TP / (TP + FN)         # sensitivity, recall, hit rate, or true positive rate (TPR)
                TNR = TN / (TN + FP)          # specificity, selectivity or true negative rate (TNR)

                precision = TP / (TP + FP)    # precision or positive predictive value (PPV)  -> BETTER FOR IMBALANCED DATASETS WITH MORE NEGATIVE SAMPLES  

                false_positive_rate = FP / (FP + TN)      # fall-out or false positive rate (FPR)

                Fscore = (2*TP) / (2*TP + FP + FN)      # F1 score - is the harmonic mean of precision and sensitivity:



              #results_dict[fft_iter][filt_iter][coef_iter][kern_iter][offset_iter][mfilter_iter] = [precision, recall, TNR, false_positive_rate]
              results_df.loc[len(results_df), :] = [fft_iter, filt_iter, coef_iter, kern_iter, 
                                            offset_iter, mfilter_iter, precision, recall, 
                                            TNR, false_positive_rate, Fscore, [TP, TN, FN, FP]]
              
              # Graphical control for long iterations
              a += 1
              actual_time = time.time()

              sys.stdout.write('\rIteration ' + str(a) + '.\tLast iteration time: ' + str(actual_time - start_time))

              start_time = actual_time
              
        # Force python to free RAM after each exec
        del D
        del SSM
        gc.collect()





In [ ]:
# THIS SCRIPT REPRESENTS ONE LOOP OF OPTIMITZATION

def showResults():
  max_fscore = results_df.loc[results_df['F-score'].astype('float').idxmax()]
  print('PARAMETERS FOR MAX VALUE OF F-SCORE: ')
  print(max_fscore)
  plot_param = (max_fscore)[0:6].tolist()
  print('\nREPRESENTATION OF THIS SIMULATION:')
  fm.Foote(*plot_param)
  return plot_param


looptime = time.time()
print('\n')

R_FFT_size = [ 1024*2**3] 
FFT_size = [(1024*2**3)]

R_num_mel_filt =range(20, 60, 2)
num_mel_filt = [40]

R_num_mel_coefficients = range(8, 32, 2)
num_mel_coefficients = [20]

R_kernel_size = range(180, 320, 2)
kernel_size = [250]

R_offset = np.arange(0.3, 0.7, 0.005)
offset = [0.45]

R_median_filter_size = range(35, 95, 1)
median_filter_size = [55]


Foote(R_FFT_size, num_mel_filt, num_mel_coefficients, kernel_size, offset, median_filter_size)
print('\n\n')
optim = showResults()
FFT_size = [optim[0]]
print('Optimization time: ' + str(time.time() - looptime))
print('\n\n\n')


Foote(FFT_size, R_num_mel_filt, num_mel_coefficients, kernel_size, offset, median_filter_size)
print('\n\n')
optim = showResults()
num_mel_filt = [optim[1]]
print('Optimization time: ' + str(time.time() - looptime))
print('\n\n\n')

Foote(FFT_size, num_mel_filt, R_num_mel_coefficients, kernel_size, offset, median_filter_size)
print('\n\n')
optim = showResults()
num_mel_coefficients = [optim[2]]
print('Optimization time: ' + str(time.time() - looptime))
print('\n\n\n')

Foote(FFT_size, num_mel_filt, num_mel_coefficients, R_kernel_size, offset, median_filter_size)
print('\n\n')
optim = showResults()
kernel_size = [optim[3]]
print('Optimization time: ' + str(time.time() - looptime))
print('\n\n\n')

Foote(FFT_size, num_mel_filt, num_mel_coefficients, kernel_size, R_offset, median_filter_size)
print('\n\n')
optim = showResults()
offset = [optim[4]]
print('Optimization time: ' + str(time.time() - looptime))
print('\n\n\n')

Foote(FFT_size, num_mel_filt, num_mel_coefficients, kernel_size, offset, R_median_filter_size)
print('\n\n')
optim = showResults()
median_filter_size = [optim[5]]
print('Optimization time: ' + str(time.time() - looptime))
print('\n\n\n')



In [ ]:
# Saving the results in a csv file
results_df.to_csv('/content/drive/MyDrive/TFG/opti_results.csv', sep=',')

In [ ]:
# Custom trash collector
del D
del SSM
gc.collect()

In [ ]:
results_df

,FFT size,N Mel Filters,N Mel Coefficients,Size Kernel,Offset in filter,Median filter size,Precision,Recall,Specificity,False Positive Rate,F-score,Confusion Matrix
0,4096,60,25,250,0.45,55,0.688889,0.516667,0.987261,0.012739,0.590476,"[31, 1085, 29, 14]"
1,8192,60,25,250,0.45,55,0.72093,0.516667,0.989081,0.010919,0.601942,"[31, 1087, 29, 12]"
2,16384,60,25,250,0.45,55,0.710526,0.45,0.989991,0.010009,0.55102,"[27, 1088, 33, 11]"
3,8192,30,25,250,0.45,55,0.681818,0.5,0.987261,0.012739,0.576923,"[30, 1085, 30, 14]"
4,8192,40,25,250,0.45,55,0.711111,0.533333,0.988171,0.011829,0.609524,"[32, 1086, 28, 13]"
...,...,...,...,...,...,...,...,...,...,...,...,...
87,8192,40,24,250,0.45,55,0.711111,0.533333,0.988171,0.011829,0.609524,"[32, 1086, 28, 13]"
88,8192,40,26,250,0.45,55,0.711111,0.533333,0.988171,0.011829,0.609524,"[32, 1086, 28, 13]"
89,8192,40,28,250,0.45,55,0.711111,0.533333,0.988171,0.011829,0.609524,"[32, 1086, 28, 13]"
90,8192,40,30,250,0.45,55,0.711111,0.533333,0.988171,0.011829,0.609524,"[32, 1086, 28, 13]"
